---
title: 15.1.1. TP - Déploiement de WordPress avec Docker Compose 📃
date: 2025-09-13
authors:
  - name: Chahrazed Labba
    email: chahrazed.labba@univ-lorraine.fr
    affiliations:
      - name: Université de Lorraine
        ror: 04vfs2w97
---

# **15.1.1. TP - Déploiement de WordPress avec Docker Compose**

# **Introduction**

Dans ce TP, nous allons déployer localement WordPress sur votre ordinateur en utilisant des conteneurs et un fichier docker-compose.yml. Ce cas est une véritable utilisation de Docker : avant de déployer un logiciel dans une organisation, on peut le tester en local pour vérifier qu’il répond bien aux besoins.

## **Qu’est-ce que WordPress ?**

WordPress est un CMS (Content Management System) open-source, utilisé pour créer des sites web et des blogs sans coder.
- Interface conviviale, thèmes et plugins
- Gestion flexible du contenu
- Optimisé pour le référencement
- Largement adopté (43% des sites web en 2023)

# **Objectifs de l’architecture**

D’après la documentation, il nous faudra :
- un conteneur WordPress
- un conteneur pour la base de données (MariaDB)
- des volumes pour stocker :
  - les plugins, thèmes et fichiers WordPress
  - les données des utilisateurs (comptes, contenu, etc.)
- des variables d’environnement pour connecter WordPress à la base
- une politique de redémarrage en cas de plantage

# **Étape 1 : Préparation du projet**

Commencez par créer un **répertoire de travail** dédié à ce TP, par exemple `mon-wordpress`. Placez-vous ensuite dans ce répertoire, puis créez un fichier `docker-compose.yml` qui
contiendra la description de l’architecture multi-conteneurs.
Au départ, définissez simplement deux services vides :
```yml
services:
    db:
    wordpress:
```

# **Étape 2 : Choix des images**

```yml
services:
    db:
        image: mariadb:10.6.4-focal
    wordpress:
        image: wordpress:latest
```

# **Étape 3 : Communication entre conteneurs**

```yml
services:
    db:
        image: mariadb:10.6.4-focal
        expose:
            - 3306
            - 33060
    wordpress:
        image: wordpress:latest
        ports:
            - "80:80"
```
`expose` : communication interne entre conteneurs  
`ports` : ccès externe à WordPress depuis l’hôte

# **Étape 4 : Ajout de volumes**

```yml
services:
    db:
        image: mariadb:10.6.4-focal
        expose:
            - 3306
            - 33060
        volumes:
            - b_data:/var/lib/mysql
    wordpress:
        image: wordpress:latest
        ports:
            - "80:80"
        volumes:
            - wp_data:/var/www/html

volumes:
    db_data:
    wp_data:
```

# **Étape 5 : Variables d’environnement**

```yml
services:
    db:
        image: mariadb:10.6.4-focal
        expose:
            - 3306
            - 33060
        volumes:
            - b_data:/var/lib/mysql
        environment:
            - MYSQL_ROOT_PASSWORD=somewordpress
            - MYSQL_DATABASE=wordpress
            - MYSQL_USER=wordpress
            - MYSQL_PASSWORD=wordpress
    wordpress:
        image: wordpress:latest
        ports:
            - "80:80"
        volumes:
            - wp_data:/var/www/html
        environment:
            - WORDPRESS_DB_HOST=db
            - WORDPRESS_DB_USER=wordpress
            - WORDPRESS_DB_PASSWORD=wordpress
            - WORDPRESS_DB_NAME=wordpress

volumes:
    db_data:
    wp_data:
```

# **Étape 6 : Politique de redémarrage**

Il est recommandé de préciser une politique de redémarrage pour chaque service. Ajoutez la ligne suivante dans la définition des services db et wordpress :
```yml
restart: always
```

```yml
services:
    db:
        image: mariadb:10.6.4-focal
        expose:
            - 3306
            - 33060
        volumes:
            - b_data:/var/lib/mysql
        environment:
            - MYSQL_ROOT_PASSWORD=somewordpress
            - MYSQL_DATABASE=wordpress
            - MYSQL_USER=wordpress
            - MYSQL_PASSWORD=wordpress
        restart: always
    wordpress:
        image: wordpress:latest
        ports:
            - "80:80"
        volumes:
            - wp_data:/var/www/html
        environment:
            - WORDPRESS_DB_HOST=db
            - WORDPRESS_DB_USER=wordpress
            - WORDPRESS_DB_PASSWORD=wordpress
            - WORDPRESS_DB_NAME=wordpress
        restart: always

volumes:
    db_data:
    wp_data:
```

# **Étape 7 : Paramètre d’authentification MariaDB**

Par défaut, MariaDB peut utiliser différents plugins d’authentification pour gérer les utilisateurs. Certaines versions récentes activent le plugin `caching_sha2_password`, qui
n’est pas toujours compatible avec WordPress et peut entraîner des erreurs de connexion
(par exemple : `"authentication method unknown to the client"`).  

Pour garantir la compatibilité, nous allons forcer MariaDB à utiliser le plugin d’authentification classique `mysql_native_password`. Cela se fait grâce à la directive `command:`, qui permet de modifier la commande de démarrage du conteneur.  

Ajoutez donc la ligne suivante dans la configuration du service db :  
`command: '--default-authentication-plugin=mysql_native_password'`

```yml
services:
    db:
        image: mariadb:10.6.4-focal
        expose:
            - 3306
            - 33060
        volumes:
            - b_data:/var/lib/mysql
        environment:
            - MYSQL_ROOT_PASSWORD=somewordpress
            - MYSQL_DATABASE=wordpress
            - MYSQL_USER=wordpress
            - MYSQL_PASSWORD=wordpress
        restart: always
        command: '--default-authentication-plugin=mysql_native_password'
    wordpress:
        image: wordpress:latest
        ports:
            - "80:80"
        volumes:
            - wp_data:/var/www/html
        environment:
            - WORDPRESS_DB_HOST=db
            - WORDPRESS_DB_USER=wordpress
            - WORDPRESS_DB_PASSWORD=wordpress
            - WORDPRESS_DB_NAME=wordpress
        restart: always

volumes:
    db_data:
    wp_data:
```

# **Étape 8 : Lancement et test**

```bash
docker compose up -d
```

Aller sur http://localhost:8080 pour configurer WordPress.

![](media/images/TP01_docker_wordpress.png)

# **Étape 9 : Nettoyage**

Lister les volumes :  
```bash
docker volume ls
```
Arrêter les services :  
```bash
docker compose down
```
Supprimer les volumes créés :
```bash
docker volume rm mon-wordpress_db_data
docker volume rm mon-wordpress_wp_data
```

<picture>
  <!-- Light mode image -->
  <source srcset="media/images/Excalidraw_DOMASS_TP01_Docker_light.png" media="(prefers-color-scheme: light)" />

  <!-- Dark mode image -->
  <source srcset="media/images/Excalidraw_DOMASS_TP01_Docker_dark.png" media="(prefers-color-scheme: dark)" />

  <!-- Fallback (if prefers-color-scheme isn't supported) -->
  <img src="media/images/Excalidraw_DOMASS_TP01_Docker_light.png" alt="Description of image" />
</picture>

![](media/images/Excalidraw_DOMASS_TP01_Docker_light.png#gh-light-mode-only)
![](media/images/Excalidraw_DOMASS_TP01_Docker_dark.png#gh-dark-mode-only)

<picture>
  <!-- Source for dark mode -->
  <source media="(prefers-color-scheme: dark)" srcset="media/images/Excalidraw_DOMASS_TP01_Docker_dark.png">
  <!-- Fallback image for light mode and other clients -->
  <img alt="My Alt Text" src="media/images/Excalidraw_DOMASS_TP01_Docker_light.png">
</picture>


https://dev.to/srbhr/how-to-display-different-images-for-github-light-mode-and-dark-mode-edg

https://blog.stephane-robert.info/docs/conteneurs/orchestrateurs/docker-compose/